In [1]:
from Network import FeedForwardNN
import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
from torch.optim import Adam
import numpy as np
from MAMEToolkit.emulator import Emulator
from MAMEToolkit.emulator import Address
from MAMEToolkit.sf_environment.Actions import Actions
from Steps import *

class PPOEnv:
    def __init__(self, env):
        #Initialise hyperparameters
        self._init_hyperparameters()
        
        self.render = True
        self.frame_skip = False
        self.sound = False
        self.debug = False
        self.binary_path = None
        self.difficulty = 3
        self.throttle = throttle
        self.started = False
        self.expected_health = {"P1": 0, "P2": 0}
        self.expected_wins = {"P1": 0, "P2": 0}
        self.round_done = False
        self.stage_done = False
        self.game_done = False
        self.stage = 1
        self.emu = Emulator(env_id, roms_path, "sfiii3n", setup_memory_addresses(), frame_ratio=frame_ratio, render=render, throttle=throttle, frame_skip=frame_skip, sound=sound, debug=debug, binary_path=binary_path)

        #Extract environment info
        self.env = env
        self.obs_dim = env.observation_space.shape[0]
        self.act_dim = env.action_space.shape[0]
        
        #ALG STEP 1
        #Initialise actor and critic networks
        self.actor = FeedForwardNN(self.obs_dim, self.act_dim)
        self.critic = FeedForwardNN(self.obs_dim, 1)
        
        #Variable for covariance matrix
        self.cov_var = torch.full(size = (self.act_dim,), fill_value = 0.5)
        
        #Create covariance matrix
        self.cov_mat = torch.diag(self.cov_var)
        
        #Initialise Adam optimiser for actor and critic
        self.actor_optim = Adam(self.actor.parameters(), lr=self.lr)
        self.critic_optim = Adam(self.critic.parameters(), lr=self.lr)
        
    def _init_hyperparameters(self):
        self.max_timesteps_per_batch = 4800
        self.max_timesteps_per_episode = 1600
        self.gamma = 0.95
        self.epochs = 5
        self.clip = 0.2 #Recommended value from PPO original paper
        self.lr = 0.005
        
        self.frame_ratio = 2
        self.frames_per_step = 2
        
    def get_action(self, obs):
        #Get mean from actor (Identical to self.actor.forward(obs))
        mean = self.actor(obs)
        
        #Create Multivariate Normal Distribution
        dist = MultivariateNormal(mean, self.cov_mat)
        
        #Sample action from the distribution
        action = dist.sample()
        
        #Get log prob from action from distribution
        log_prob = dist.log_prob(action)
        
        #Isolate tensors from graphs and return action and log_prob
        return action.detach().numpy(), log_prob.detach()
        
    def compute_rtgs(self, batch_rews):
        #Rewards-to-go per episode per batch to return
        #Shape will be (number of timesteps per episode)
        batch_rtgs = []
        
        #Interate through episodes backwards to maintain same order in batch_rtgs
        for ep_rews in reversed(batch_rews):
            discounted_reward = 0 #Discounted reward so far
            
            for rew in reversed(ep_rews):
                discounted_reward = rew + discounted_reward * self.gamma
                batch_rtgs.insert(0, discounted_reward)
                
        #Convert rewards-to-go to tensor
        batch_rtgs = torch.tensor(batch_rtgs, dtype = torch.float)
        
        return batch_rtgs
        
    def learn(self, t_max):
        #Timesteps simulated so far
        t_now = 0
        
        #ALG STEP 2
        while t_now < t_max:
            #ALG STEP 3
            batch_obs, batch_acts, batch_log_probs, batch_rtgs, batch_lens = self.rollout()
            
            #Increment current timesteps by timesteps gained this batch
            t_now += np.sum(batch_lens)
            
            #Calculate V_(phi, k)
            V, _ = self.evaluate(batch_obs, batch_acts)
            
            #ALG STEP 5
            #Calculate advantage
            A_k = batch_rtgs - V.detach()
            
            #Normalize advantage
            A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-10) #add 1e-10 to avoid divide-by-zero
            
            for _ in range(self.epochs): #ALG STEP 6 & 7
                #Calculate V_phi and pi_theta(a_t | s_t)
                V, curr_log_probs = self.evaluate(batch_obs, batch_acts)
                
                #Calculate ratios
                ratios = torch.exp(curr_log_probs - batch_log_probs)
                
                #Calculate surrogate losses
                surr1 = ratios * A_k #Raw ratios
                surr2 = torch.clamp(ratios, 1 - self.clip, 1 + self.clip) * A_k #Binds ratios within clip hyperparameter from 1

                #Negative minimum of losses causes Adam optimiser to maximise loss
                #Then get single loss by getting the mean
                actor_loss = (-torch.min(surr1, surr2)).mean()
                
                #Calculate critic loss as mean-squared error of predictions and rewards-to-go
                critic_loss = nn.MSELoss()(V, batch_rtgs)
                
                #Calculate gradients and perform backpropagation on actor network
                self.actor_optim.zero_grad()
                actor_loss.backward(retain_graph = True)
                self.actor_optim.step()
                
                #Calculate gradients and perform backpropagation on critic network
                self.critic_optim.zero_grad()
                critic_loss.backward()
                self.critic_optim.step()
            
    def evaluate(self, batch_obs, batch_acts):
        #Get predictions V for each obs in batch_obs from critic network, squeezing to reduce tensor dimensions to 1
        V = self.critic(batch_obs).squeeze()
        
        #Calculate log probabilites of batch actions using most recent actor network
        mean = self.actor(batch_obs)
        dist = MultivariateNormal(mean, self.cov_mat)
        log_probs = dist.log_prob(batch_acts)
        
        #Return predictions V and log probs log_probs
        return V, log_probs
            
    def rollout(self):
        #Batch data
        batch_obs = [] #observations
        batch_acts = [] #actions
        batch_log_probs = [] #log probabilities of each action
        batch_rews = [] #rewards
        batch_rtgs = [] #rewards-to-go
        batch_lens = [] #episodic lengths in batch
        
        #Timesteps simulated so far
        t_now = 0
        
        while t_now < self.max_timesteps_per_batch:
            #Rewards this episode
            ep_rews = []
            
            obs = self.env.reset()
            done = False
            
            for ep_t in range(self.max_timesteps_per_episode):
                #Increment timesteps ran this batch so far
                t_now += 1
                
                #Collect observation
                batch_obs.append(obs)
                
                action, log_prob = self.get_action(obs)
                obs, rew, done, _ = self.env.step(action)
                
                #Collect reward, action and log prob
                ep_rews.append(rew)
                batch_acts.append(action)
                batch_log_probs.append(log_prob)
                
                if done:
                    break
                    
            #Collect episode length and reward
            batch_lens.append(ep_t + 1)
            batch_rews.append(ep_rews)
        
        #Reshape data as tensors for drawing computation graphs
        batch_obs = torch.tensor(batch_obs, dtype=torch.float)
        batch_acts = torch.tensor(batch_acts, dtype=torch.float)
        batch_log_probs = torch.tensor(batch_log_probs, dtype=torch.float)
        
        #ALG STEP 4
        batch_rtgs = self.compute_rtgs(batch_rews)
        
        #Return batch data
        return batch_obs, batch_acts, batch_log_probs, batch_rtgs, batch_lens

    # Runs a set of action steps over a series of time steps
    # Used for transitioning the emulator through non-learnable gameplay, aka. title screens, character selects
    def run_steps(self, steps):
        for step in steps:
            for i in range(step["wait"]):
                self.emu.step([])
            self.emu.step([action.value for action in step["actions"]])

    # Must be called first after creating this class
    # Sends actions to the game until the learnable gameplay starts
    # Returns the first few frames of gameplay
    def start(self):
        if self.throttle:
            for i in range(int(250/self.frame_ratio)):
                self.emu.step([])
        self.run_steps(set_difficulty(self.frame_ratio, self.difficulty))
        self.run_steps(start_game(self.frame_ratio))
        frames = self.wait_for_fight_start()
        self.started = True
        return frames

    # Observes the game and waits for the fight to start
    def wait_for_fight_start(self):
        data = self.emu.step([])
        while data["fighting"] == 0:
            data = self.emu.step([])
        self.expected_health = {"P1": data["healthP1"], "P2": data["healthP2"]}
        data = self.gather_frames([])
        return data["frame"]

    def reset(self):
        if self.game_done:
            return self.new_game()
        elif self.stage_done:
            return self.next_stage()
        elif self.round_done:
            return self.next_round()
        else:
            raise EnvironmentError("Reset called while gameplay still running")

    # To be called when a round finishes
    # Performs the necessary steps to take the agent to the next round of gameplay
    def next_round(self):
        self.round_done = False
        self.expected_health = {"P1": 0, "P2": 0}
        return self.wait_for_fight_start()

    # To be called when a game finishes
    # Performs the necessary steps to take the agent(s) to the next game and resets the necessary book keeping variables
    def next_stage(self):
        self.wait_for_continue()
        self.run_steps(next_stage(self.frame_ratio))
        self.expected_health = {"P1": 0, "P2": 0}
        self.expected_wins = {"P1": 0, "P2": 0}
        self.round_done = False
        self.stage_done = False
        return self.wait_for_fight_start()

    def new_game(self):
        self.wait_for_continue()
        self.run_steps(new_game(self.frame_ratio))
        self.expected_health = {"P1": 0, "P2": 0}
        self.expected_wins = {"P1": 0, "P2": 0}
        self.round_done = False
        self.stage_done = False
        self.game_done = False
        self.stage = 1
        return self.wait_for_fight_start()

    # Steps the emulator along until the screen goes black at the very end of a game
    def wait_for_continue(self):
        data = self.emu.step([])
        if self.frames_per_step == 1:
            while data["frame"].sum() != 0:
                data = self.emu.step([])
        else:
            while data["frame"][0].sum() != 0:
                data = self.emu.step([])

    # Steps the emulator along until the round is definitely over
    def run_till_victor(self, data):
        while self.expected_wins["P1"] == data["winsP1"] and self.expected_wins["P2"] == data["winsP2"]:
            data = add_rewards(data, self.sub_step([]))
        self.expected_wins = {"P1":data["winsP1"], "P2":data["winsP2"]}
        return data

    # Checks whether the round or game has finished
    def check_done(self, data):
        if data["fighting"] == 0:
            data = self.run_till_victor(data)
            self.round_done = True
            if data["winsP1"] == 2:
                self.stage_done = True
                self.stage += 1
            if data["winsP2"] == 2:
                self.game_done = True
        return data

    # Collects the specified amount of frames the agent requires before choosing an action
    def gather_frames(self, actions):
        data = self.sub_step(actions)
        frames = [data["frame"]]
        for i in range(self.frames_per_step - 1):
            data = add_rewards(data, self.sub_step(actions))
            frames.append(data["frame"])
        data["frame"] = frames[0] if self.frames_per_step == 1 else frames
        return data

    # Steps the emulator along by one time step and feeds in any actions that require pressing
    # Takes the data returned from the step and updates book keeping variables
    def sub_step(self, actions):
        data = self.emu.step([action.value for action in actions])

        p1_diff = (self.expected_health["P1"] - data["healthP1"])
        p2_diff = (self.expected_health["P2"] - data["healthP2"])
        self.expected_health = {"P1": data["healthP1"], "P2": data["healthP2"]}

        rewards = {
            "P1": (p2_diff-p1_diff),
            "P2": (p1_diff-p2_diff)
        }

        data["rewards"] = rewards
        return data

    # Steps the emulator along by the requested amount of frames required for the agent to provide actions
    def step(self, move_action, attack_action):
        if self.started:
            if not self.round_done and not self.stage_done and not self.game_done:
                actions = []
                actions += index_to_move_action(move_action)
                actions += index_to_attack_action(attack_action)
                data = self.gather_frames(actions)
                data = self.check_done(data)
                return data["frame"], data["rewards"], self.round_done, self.stage_done, self.game_done
            else:
                raise EnvironmentError("Attempted to step while characters are not fighting")
        else:
            raise EnvironmentError("Start must be called before stepping")

    # Safely closes emulator
    def close(self):
        self.emu.close()